In [ ]:
import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import folium

import networkx as nx

In [77]:
traffic_flow = gpd.read_file("D:\Datasets\MIMOS/kl_selangor_traffic_flow")
traffic_flow.head()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,geometry
0,4712,E6/LEBUHRAYA UTARA SELATAN HUBUNGAN TENGAH (NO...,-,0.58626,1,50.0,50.0,50.0,0.00000,0.7,2020-08-26T07:59:35Z,None,"LINESTRING (101.56610 3.05161, 101.56549 3.051..."
1,4669,2/Jalan Parang/Jalan Kurau/Jalan Pelabuhan Utara,-,0.09153,4,33.0,33.0,43.0,2.17054,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.37124 3.01689, 101.37123 3.016..."
2,4670,Lebuh Sultan Mohamed 1 (West),-,1.88608,2,68.0,68.0,77.3,1.60413,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.37113 3.01771, 101.37103 3.018..."
3,4671,Jalan Sultan Abdul Samad 40,-,0.63894,2,72.0,72.0,85.7,2.13146,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.37842 3.03190, 101.37860 3.032..."
4,4672,Lebuh Sultan Mohamed 1 (East),-,0.31368,2,72.0,72.0,84.9,2.02591,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.38230 3.03616, 101.38255 3.036..."


In [78]:
traffic_flow.dtypes

TMC_PC         int64
TMC_DE        object
TMC_QD        object
TMC_LE       float64
FC             int64
CF_SP        float64
CF_SU        float64
CF_FF        float64
CF_JF        float64
CF_CN        float64
PBT           object
DE            object
geometry    geometry
dtype: object

In [79]:
traffic_flow['endpoints'] = traffic_flow['geometry'].apply(lambda x: x.boundary)



In [86]:
def return_start_point(x):
    #print(x)
    try:
        return x.boundary[0]
    except:
        return None

def return_end_point(x):
    try:
        return x.boundary[1]
    except:
        return None
    

traffic_flow['start_point'] = traffic_flow['geometry'].apply(lambda x: return_start_point(x))
traffic_flow['end_point'] = traffic_flow['geometry'].apply(lambda x: return_end_point(x))

In [87]:
traffic_flow.head()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,geometry,endpoints,start_point,end_point
0,4712,E6/LEBUHRAYA UTARA SELATAN HUBUNGAN TENGAH (NO...,-,0.58626,1,50.0,50.0,50.0,0.00000,0.7,2020-08-26T07:59:35Z,None,"LINESTRING (101.56610 3.05161, 101.56549 3.051...","MULTIPOINT (101.56610 3.05161, 101.56548 3.05003)",POINT (101.56610 3.05161),POINT (101.56548 3.05003)
1,4669,2/Jalan Parang/Jalan Kurau/Jalan Pelabuhan Utara,-,0.09153,4,33.0,33.0,43.0,2.17054,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.37124 3.01689, 101.37123 3.016...","MULTIPOINT (101.37124 3.01689, 101.37113 3.01771)",POINT (101.37124 3.01689),POINT (101.37113 3.01771)
2,4670,Lebuh Sultan Mohamed 1 (West),-,1.88608,2,68.0,68.0,77.3,1.60413,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.37113 3.01771, 101.37103 3.018...","MULTIPOINT (101.37113 3.01771, 101.37842 3.03190)",POINT (101.37113 3.01771),POINT (101.37842 3.03190)
3,4671,Jalan Sultan Abdul Samad 40,-,0.63894,2,72.0,72.0,85.7,2.13146,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.37842 3.03190, 101.37860 3.032...","MULTIPOINT (101.37842 3.03190, 101.38230 3.03616)",POINT (101.37842 3.03190),POINT (101.38230 3.03616)
4,4672,Lebuh Sultan Mohamed 1 (East),-,0.31368,2,72.0,72.0,84.9,2.02591,0.7,2020-08-26T07:59:35Z,Lebuhraya Selat Klang,"LINESTRING (101.38230 3.03616, 101.38255 3.036...","MULTIPOINT (101.38230 3.03616, 101.38424 3.03822)",POINT (101.38230 3.03616),POINT (101.38424 3.03822)


In [88]:
traffic_flow.to_csv('D:\Datasets\MIMOS/kl_selangor_traffic_flow_v2.csv', index=False)

In [73]:
for row in traffic_flow.iterrows():
    #print(row[1]['geometry'])
    print(row[1]['endpoints'])
    print(row[1]['endpoints'][0])
    print(row[1]['endpoints'][1])

MULTIPOINT (101.5661 3.05161, 101.56548 3.05003)
POINT (101.5661 3.05161)
POINT (101.56548 3.05003)
MULTIPOINT (101.37124 3.01689, 101.37113 3.01771)
POINT (101.37124 3.01689)
POINT (101.37113 3.01771)
MULTIPOINT (101.37113 3.01771, 101.37842 3.0319)
POINT (101.37113 3.01771)
POINT (101.37842 3.0319)
MULTIPOINT (101.37842 3.0319, 101.3823 3.03616)
POINT (101.37842 3.0319)
POINT (101.3823 3.03616)
MULTIPOINT (101.3823 3.03616, 101.38424 3.03822)
POINT (101.3823 3.03616)
POINT (101.38424 3.03822)
MULTIPOINT (101.38424 3.03822, 101.38733 3.04157)
POINT (101.38424 3.03822)
POINT (101.38733 3.04157)
MULTIPOINT (101.38733 3.04157, 101.39728 3.04993)
POINT (101.38733 3.04157)
POINT (101.39728 3.04993)
MULTIPOINT (101.39728 3.04993, 101.40732 3.05625)
POINT (101.39728 3.04993)
POINT (101.40732 3.05625)
MULTIPOINT (101.40732 3.05625, 101.4133 3.05999)
POINT (101.40732 3.05625)
POINT (101.4133 3.05999)
MULTIPOINT (101.4133 3.05999, 101.42439 3.06498)
POINT (101.4133 3.05999)
POINT (101.42439 3.0

IndexError: tuple index out of range

traffic_flow['start_point'] = traffic_flow['endpoints'].apply(lambda x: str(x).split(',')[0].replace("MULTIPOINT(",''))
#traffic_flow['test'] = traffic_flow['geometry'].apply(lambda x: x.coords[0])
traffic_flow['end_point'] = traffic_flow['endpoints'].apply(lambda x: str(x).split(',')[-1].replace(')',''))
traffic_flow['end_point_x'] = traffic_flow['end_point'].apply(lambda x: x.split('\\n')[0])